In [19]:
import glob, os
import pandas as pd

In [2]:
transl_path = "/home/craig.car/repos/chiron/align_texts_project/data/translations_repositories.csv"

In [47]:
transl_df = pd.read_csv(transl_path)
transl_df = transl_df.rename(columns={"CTS-URN/s":"cts-urns_grp", "Original File Name/Location":"Original_File_Name", "Addiional File location/locations":"Additional_File_locations"})

In [48]:
transl_df.loc[0,"cts-urns_grp"]

'urn:cts:greekLit:tlg0085.tlg001.ogl-eng1 urn:cts:greekLit:tlg0085.tlg002.ogl-eng1 urn:cts:greekLit:tlg0085.tlg003.ogl-eng1 urn:cts:greekLit:tlg0085.tlg004.ogl-eng1 urn:cts:greekLit:tlg0085.tlg005.ogl-eng1 urn:cts:greekLit:tlg0085.tlg006.ogl-eng1 urn:cts:greekLit:tlg0085.tlg007.ogl-eng1'

In [49]:
transl_df.loc[0,"cts-urns_grp"].split()

['urn:cts:greekLit:tlg0085.tlg001.ogl-eng1',
 'urn:cts:greekLit:tlg0085.tlg002.ogl-eng1',
 'urn:cts:greekLit:tlg0085.tlg003.ogl-eng1',
 'urn:cts:greekLit:tlg0085.tlg004.ogl-eng1',
 'urn:cts:greekLit:tlg0085.tlg005.ogl-eng1',
 'urn:cts:greekLit:tlg0085.tlg006.ogl-eng1',
 'urn:cts:greekLit:tlg0085.tlg007.ogl-eng1']

In [50]:
transl_df["cts-urns_uniques"] = transl_df["cts-urns_grp"].str.split()

In [51]:
transl_df = transl_df.explode("cts-urns_uniques", ignore_index=True)

In [52]:
transl_df["ctsurns"] = transl_df["cts-urns_uniques"].str.rsplit(".",1).str[0]

/tmp/ipykernel_8780/1713395687.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  transl_df["ctsurns"] = transl_df["cts-urns_uniques"].str.rsplit(".",1).str[0]


In [53]:
transl_df["filename1"] = transl_df["Original_File_Name"].str.split(".").str[0]

In [54]:
transl_df["filename2"] = transl_df["Additional_File_locations"].str.split(".").str[0]

In [55]:
transl_df.head()

,Author Name,WORK IDs,cts-urns_grp,# of translations,Edition Title,Editor; Translator,Publisher,Date,Original_File_Name,Additional_File_locations,...,Worldcat Link,Work/s Split Up/Renamed in Branch,New File Name/s,Notes,TOC,Additional TOC,cts-urns_uniques,ctsurns,filename1,filename2
0,Aeschylus,0085.001 0085.002 0085.003 0085.004 0085.005 0...,urn:cts:greekLit:tlg0085.tlg001.ogl-eng1 urn:c...,7.0,The Dramas of Aeschylus,"Swanwick, Anna, 1831-1899",Bell,1886,aeschylus_1886.xml,NaN,...,NaN,no,NaN,NaN,NaN,NaN,urn:cts:greekLit:tlg0085.tlg001.ogl-eng1,urn:cts:greekLit:tlg0085.tlg001,aeschylus_1886,NaN
1,Aeschylus,0085.001 0085.002 0085.003 0085.004 0085.005 0...,urn:cts:greekLit:tlg0085.tlg001.ogl-eng1 urn:c...,7.0,The Dramas of Aeschylus,"Swanwick, Anna, 1831-1899",Bell,1886,aeschylus_1886.xml,NaN,...,NaN,no,NaN,NaN,NaN,NaN,urn:cts:greekLit:tlg0085.tlg002.ogl-eng1,urn:cts:greekLit:tlg0085.tlg002,aeschylus_1886,NaN
2,Aeschylus,0085.001 0085.002 0085.003 0085.004 0085.005 0...,urn:cts:greekLit:tlg0085.tlg001.ogl-eng1 urn:c...,7.0,The Dramas of Aeschylus,"Swanwick, Anna, 1831-1899",Bell,1886,aeschylus_1886.xml,NaN,...,NaN,no,NaN,NaN,NaN,NaN,urn:cts:greekLit:tlg0085.tlg003.ogl-eng1,urn:cts:greekLit:tlg0085.tlg003,aeschylus_1886,NaN
3,Aeschylus,0085.001 0085.002 0085.003 0085.004 0085.005 0...,urn:cts:greekLit:tlg0085.tlg001.ogl-eng1 urn:c...,7.0,The Dramas of Aeschylus,"Swanwick, Anna, 1831-1899",Bell,1886,aeschylus_1886.xml,NaN,...,NaN,no,NaN,NaN,NaN,NaN,urn:cts:greekLit:tlg0085.tlg004.ogl-eng1,urn:cts:greekLit:tlg0085.tlg004,aeschylus_1886,NaN
4,Aeschylus,0085.001 0085.002 0085.003 0085.004 0085.005 0...,urn:cts:greekLit:tlg0085.tlg001.ogl-eng1 urn:c...,7.0,The Dramas of Aeschylus,"Swanwick, Anna, 1831-1899",Bell,1886,aeschylus_1886.xml,NaN,...,NaN,no,NaN,NaN,NaN,NaN,urn:cts:greekLit:tlg0085.tlg005.ogl-eng1,urn:cts:greekLit:tlg0085.tlg005,aeschylus_1886,NaN


In [56]:
transl_df["ctsurns"].duplicated().sum()

796

In [67]:
test = list(transl_df.loc[transl_df["ctsurns"] == "urn:cts:greekLit:tlg0006.tlg001", "filename1"])

In [68]:
test.extend(list(transl_df.loc[transl_df["ctsurns"] == "urn:cts:greekLit:tlg0006.tlg001", "filename2"]))

In [69]:
test

['euripides_1_1884',
 'euripides_1857_1',
 'euripides_1906_3',
 'euripides_1911',
 'euripides_2_1884',
 nan,
 nan,
 nan]

In [63]:
transl_df.loc[transl_df["ctsurns"] == "urn:cts:greekLit:tlg0006.tlg001", "filename2"]

485    euripides_2_1884
493                 NaN
494                 NaN
500                 NaN
Name: filename2, dtype: object

In [87]:
import math

In [90]:
lookup_table = {}
keys = transl_df["ctsurns"].unique()
for ctsurn in keys:
    # lookup_table[ctsurn] = list(transl_df.loc[transl_df["ctsurns"] == ctsurn, "filename"])
    vals = list(transl_df.loc[transl_df["ctsurns"] == ctsurn, "filename1"])
    for other_name in list(transl_df.loc[transl_df["ctsurns"] == ctsurn, "filename2"]):
        if not math.isnan(other_name):
            vals.append(other_name)
    lookup_table[ctsurn] = vals

TypeError: must be real number, not str

In [79]:
# spot check dict values
transl_df.loc[transl_df["ctsurns"] == "urn:cts:greekLit:tlg0085.tlg004", "filename1"]

3     aeschylus_1886
10    aeschylus_1888
17    aeschylus_1909
27    aeschylus_1872
37    aeschylus_1868
Name: filename1, dtype: object

In [85]:
type(lookup_table["urn:cts:greekLit:tlg0006.tlg001"][5])

float

In [73]:
len("/scratch/craig.car/src_data/")

28

In [74]:
"/scratch/craig.car/src_data/urn:cts:greekLit:tlg0001.tlg001"[28:]

'urn:cts:greekLit:tlg0001.tlg001'

In [76]:
params = {
    'QUEUE':'short',
    'SWORKERS':2,
    'SMEM':4
}

labse_vec_cpu = 'srun --partition=short --nodes=1 --pty --mem=4GB --time=01:00:00 /bin/bash -c "source /home/craig.car/miniconda3/bin/activate; conda activate labse_vec_pipeline; {command};"'

src_dir = "/scratch/craig.car/src_data/"
fr_dir = "/scratch/craig.car/french_trans-dev/"

for src_emb in glob.iglob(src_dir+"*.emb"):
    ctsurn = os.path.splitext(os.path.basename(src_emb))[0]
    src_prefix = src_dir+ctsurn
    tgt_files = lookup_table[ctsurn]
    for tgt_name in tgt_files:
        tgt_prefix = fr_dir+tgt_name
        # print(tgt_name+".emb")
        # check if a fr translation exists
        if os.path.isfile(tgt_prefix+".emb") != False: 
            params['command'] = f'./vecalign.py --alignment_max_size 8 --src {src_prefix+".sents"} --tgt {tgt_prefix+".sents"} --src_embed {src_prefix+".overlaps"} {src_emb} --tgt_embed {tgt_prefix+".overlaps"} {tgt_prefix+".emb"}'
            print(labse_vec_cpu.format(**params))
            # subprocess.run(slurm_run.format(**params),shell=True)
        break
            

srun --partition=short --nodes=1 --pty --mem=4GB --time=01:00:00 /bin/bash -c "source /home/craig.car/miniconda3/bin/activate; conda activate labse_vec_pipeline; ./vecalign.py --alignment_max_size 8 --src /scratch/craig.car/src_data/urn:cts:greekLit:tlg0005.tlg003.sents --tgt /scratch/craig.car/french_trans-dev/theocritus_1913.sents --src_embed /scratch/craig.car/src_data/urn:cts:greekLit:tlg0005.tlg003.overlaps /scratch/craig.car/src_data/urn:cts:greekLit:tlg0005.tlg003.emb --tgt_embed /scratch/craig.car/french_trans-dev/theocritus_1913.overlaps /scratch/craig.car/french_trans-dev/theocritus_1913.emb;"
srun --partition=short --nodes=1 --pty --mem=4GB --time=01:00:00 /bin/bash -c "source /home/craig.car/miniconda3/bin/activate; conda activate labse_vec_pipeline; ./vecalign.py --alignment_max_size 8 --src /scratch/craig.car/src_data/urn:cts:greekLit:tlg0006.tlg001.sents --tgt /scratch/craig.car/french_trans-dev/euripides_1_1884.sents --src_embed /scratch/craig.car/src_data/urn:cts:greek